# Herramientas de computación científica con SciPy 

<center> <img src="https://drive.google.com/uc?export=view&id=1QpoDoKHdtIqY4OWLL7dbv4M6YfiVrSER" alt="image info" width="400"/> </center>

"*SciPy (pronounced “Sigh Pie”) is a Python-based ecosystem of open-source software for mathematics, science, and engineering.*" —http://www.scipy.org
       
    
* Contiene funciones que ayudan a abordar problemas comunes de la computación científica.
* Trabaja en conjunto con NumPy.
* Entre sus módulos están:
    
|Nombre|Funciones relacionadas|
| :---: | :---: |
|misc|Funciones no clasificadas|
|special|Funciones especiales|
|fftpack|Transformada de Fourier|
|integrate|Integración y ecuaciones diferenciales|
|interpolate|Interpolación|
|io|Input/Output, entrada y salida de datos|
|linalg|Álgebra lineal|
|ndimage|Imágenes n-dimensionales|
|optimize|Optimización|
|signal|Procesamiento de señales|
|stats|Estadística|
|spatial|Estructuras de datos espaciales|
|constants|Constante matemáticas y físicas|
    
### Antes de implementar una rutina en su código, verifique si no existe ya una implementación en SciPy.

- Los submódulos son independientes entre si. 

# Módulo scipy.linalg: Álgebra lineal
    
- Tiene operaciones *built-in* para trabajar con matrices.
- Posee métodos para calcular productos matriciales, de Kronecker, internos y externos, entre otros.
- Podemos determinar los valores propios y calcular funciones trigonométricas para matrices.
- También tiene funciones para crear matrices especiales. *Ejemplo:* circulante, convolución.

In [ ]:
import numpy as np
from scipy import linalg as la

In [ ]:
matriz = np.array([[1,2],[3,4]])

In [ ]:
la.det(matriz) #Calcula el determinante de una matriz cuadrada

In [ ]:
la.inv(matriz) #Calcula la inversa de una matriz cuadrada

In [ ]:
vector = np.array([2,2,1])
la.norm(vector) #Calcula la norma del vector

También es posible calcular diferentes normas para un mismo arreglo.

In [ ]:
?la.norm

In [ ]:
vector2 = np.array([[2,2,1],[3,4,1]])
la.norm(vector2, axis= 0) #Norma a lo largo de las columnas

## Solución de sistemas de ecuaciones

Un problema común es la resolución de sistemas lineales de tipo $$Ax = b$$

donde $A$ es una matriz cuadrada y $det(A)\neq0$

Por ejemplo el sistema:

$$\begin{array}{lcl} 2x + 3y & = & 6 \\ 4x + 9y & = & 15 \end{array}$$

In [ ]:
#Primero definimos la matriz de coeficientes
matA = np.array([[2,3],[4,9]])

#Luego, definimos el vector b o matriz de resultados
vectB = np.array([6,15]).reshape(2,1)

In [ ]:
#Finalmente, hacemos uso de la función solve de linalg
la.solve(matA, vectB)

SciPy incluye otros métodos de resolución de sistemas de ecuaciones como factorización LU o el método de Cholesky. Se pueden consultar [aquí](https://docs.scipy.org/doc/scipy/reference/linalg.html#module-scipy.linalg).

## Álgebra lineal para manipular objetos

In [ ]:
from scipy import misc
import matplotlib.pyplot as plt

In [ ]:
mapache = misc.face() #Devuelve un array que corresponde a la imagen de la cara de un mapache
plt.imshow(mapache) #Comando solo para mostrar la imagen

La imagen del mapache es una matriz, veamos cuáles son sus características.

In [ ]:
print('Forma: ',mapache.shape,'\nTamaño: ',mapache.size, '\nDimensiones: ', mapache.ndim)

La forma indica que el objeto se compone de tres matrices, cada una con forma 768 x 1024. Estas tres matrices representan un canal de color RGB.

Lo que haremos será cambiar el color de la imagen a escala de grises usando la fórmula

$Gris = 0.2126R+0.7152G+0.0722B$

In [ ]:
mapache[:,:,0] #Note de una de las matrices que cada valor en la matriz representa el valor R, G o B del pixel

In [ ]:
mapache_gris = mapache.dot([0.2126, 0.7152, 0.0722]) #Multiplicamos cada canal de color por los valores de la fórmula

plt.imshow(mapache_gris,cmap='gray')

Ahora utilizaremos descomposición en valores singulares (SVD) en esta matriz para extraer las principales características y luego reconstruir la imagen a partir de ellas. 

$$U\Sigma V^T=A$$

donde $U$ y $V^T$ son matrices cuadradas y $\Sigma$ es la matriz con los valores singulares de $A$. Estos valores pueden ser utilizados como medida de la "importancia" de las características de la matriz $A$. 

En esencia, estamos usando SVD para comprimir una imagen

In [ ]:
U,s,Vt = la.svd(mapache_gris) #Singular Value Decomposition 

In [ ]:
print(s.shape,'\n',s.ndim) #Análisis de s

In [ ]:
s

s no es la matriz $\Sigma$ que buscamos, ya que debería ser una matriz diagonal. Por lo que debemos construirla con los valores de s.

In [ ]:
Sigma = np.diag(s) #Construimos una matriz con s como diagonal 
#Computacionalmente es más económico almacenar solo un vector que una matriz diagonal

Ahora que tenemos la aproximación, reconstruiremos la imagen.

In [ ]:
from matplotlib import pyplot as plt 

plt.plot(s)
plt.show()

Noten que aunque tenemos 768 valores singulares, la mayoría de ellos tienen un valor pequeño. Por eso, tendría sentido utilizar sólo algunos de ellos para reconstruir una versión más "económica"de la imagen. 

In [ ]:
k = 30 #k representa la cantidad de valores singulares en s que vale la pena utilizar en la aproximación

In [ ]:
mapache_grisR = np.matmul(np.matmul(U,Sigma[:,:k]),Vt[:k,:]) #Multiplicación de matrices en NumPy

In [ ]:
plt.imshow(mapache_grisR,cmap='gray')

## <font color='purple'>**Ejercicio**</font>

Suponga que usted quiere construir una versión no tan "económica" de la imagen y por lo tanto quiere utilizar más valores singulares de s. Si la precisión de los valores de s es 1000, no se necesitan valores más pequeños que eso, ¿de qué manera podría determinarse el valor de k? 

In [ ]:
np.where(s>1000) 

# Módulo scipy.integrate: Integración y EDO

Por motivos históricos, a la integración numérica se le conoce también como cuadratura numérica (*quadrature*).

A pesar de que hay muchos ejemplos de integrales que se pueden calcular analíticamente, en muchos casos es necesario recurrir a los métodos numéricos para aproximar una solución. 


La integración numérica en SciPy se categoriza en dos tipos:

* Rutinas que evalúan el integrando como una función de Python (*Gaussian quadrature*)
  * quad, quadrature, fixed_quad
* Rutinas que toman un arreglo con muestras del integrando en puntos dados (*Newton-Cotes*) 
  * trapz, simps, romb


### Funciones lambda o anónimas en Python

* Una función anónima es lo que este término dice, una función **sin nombre**.
* A diferencia de las funciones tradicionales, **no se usa def para definirlas** y **se escriben en una sola expresión**.
* Generalmente se usan para escribir operaciones simples.

In [ ]:
def al_cuadrado(x): #Función tradicional
    return x**2
    
al_cuadrado(2)

In [ ]:
def al_cuadrado(x): return x**2 #Escrito en una sola línea

al_cuadrado(2)

In [ ]:
lambda x: x**2 #Notación anterior simplificada, es una función lambda

In [ ]:
quad_lambda = lambda x: x**2 #Hay que guardarlo en una variable
quad_lambda(2) #Se llama de la misma forma que las funciones tradicionales

Las funciones lambda son muy útiles cuando se quiere integrar un $f(x)$.

## Integración

Suponga que quiere saber el valor de la integral $ y = \int_{0}^{1} f(x)dx $ siendo $ f(x) = \frac{4}{(1+x^2)} $.
-  La función anónima es $f(x)$.
- Del módulo **integrate** se usa **quad()** para calcular la integral simple. 

In [ ]:
from scipy import integrate

funcionPI = lambda x: 4/(1+x**2) #El primer argumento de quad debe ser un objeto "llamable" (función, método o instancia de clase)

integrate.quad(funcionPI, 0,1) # Los argumentos 2 y 3 son los límites de integración (Pueden ser +-inf)

El resultado anterior es una tupla cuya primera entrada es la solución y la segunda un número muy cercano a cero, ¿por qué? Porque quad() retorna tanto el resultado como el **error absoluto** en el valor de la integral.

In [ ]:
integrate.quadrature(funcionPI, 0, 1) #Cuadratura gaussiana

¿Cómo integraríamos usando el método de los trapezoides?

<center> <img src="https://drive.google.com/uc?export=view&id=10dDjclIqrpewye4D9EUlfS0Av7EnOiJu" alt="image info" width="350"/> </center>

Debemos calcular $f(x)$ para cada $x$.

In [ ]:
x = np.linspace(0,1,50) #Creamos una serie de puntos entre 0 y 1
y = funcionPI(x) #Determinamos y = f(x) para cada x creado

integrate.trapz(y,x) #Atención que el orden es y,x

### Solución de integrales múltiples

Así como se puede resolver una integral sencilla, **dblquad()**, **tplquad()** y **nquad()** permiten resolver integrales dobles, triples y múltiples con integrate.

In [ ]:
vol_cubo = lambda x,y,z: 1
v = integrate.tplquad(vol_cubo,0,2,0,2,0,2) #Volumen de un cubo con arista igual a 2
print(v)

## Ecuaciones diferenciales

* El módulo integrate de SciPy también incluye rutinas para la integración de ecuaciones diferenciales ordinarias (ODE).
* **solve_ivp()** es una función para resolver problemas de valor inicial en sistemas de ecuaciones diferenciales ordinarias.
* SciPy.integrate cuenta con otros *solvers* (Runge-Kutta, BDF, LSODA, entre otros).

Suponga que tiene un circuito RC. La respuesta natural de un circuito se refiere al comportamiento del circuito, sin fuentes externas de excitación.

<center> <img src="https://drive.google.com/uc?export=view&id=1f5-ChhKsoMK_H7wdmKY10d5xBNJpalv-" alt="image info" width="200"/> </center>

Aplicando la ley de corriente de Kirchhoff podemos describir el valor de la caída de potencial en el capacitor con la ecuación diferencial de primer orden:

$$\frac{dv(t)}{dt}=-\frac{1}{RC}v(t)$$

In [ ]:
from scipy import integrate
import matplotlib.pyplot as plt
import numpy as np

def modelo_RC(t,v,r,c): #r y c son parámetros del modelo
    k = 1/(r*c)
    dvdt = -k * v
    return dvdt

v0 = [15] #Valor inicial
t = [0,2] #Span de tiempo
teval = np.linspace(0,2,11) #Tiempo donde se evalúan las soluciones

sol = integrate.solve_ivp(modelo_RC,t,v0,t_eval=teval,args=[4,0.1])
sol

In [ ]:
?integrate.solve_ivp

In [ ]:
#Graficamos solamente para ver los resultados

plt.xlabel('t[s]')
plt.ylabel('Capacitor voltage [V]')
plt.plot(sol.t,sol.y.flatten()) #la solución del model es un objeto compuesto

### ¿Qué pasa si necesitamos más de una salida?

El modelo debe editarse para que produzca dos salidas.

In [ ]:
def model(t,output):
    A, B = output
    dAdt = A*B
    dBdt = A*B - B
    return dAdt, dBdt

output0 = [0.5,3] #Condiciones iniciales para A y B
t = [0,4]
teval = np.linspace(0,4,100)

sol = integrate.solve_ivp(model, t, output0, t_eval=teval)
print(sol)

# Módulo scipy.interpolate: Interpolación

* Comprende diversos métodos de interpolación. *Ejemplos*: *spline*, multivariada, suavizado.
* También tiene funciones para trabajar con interpolantes polinómicos de Lagrange y Taylor.
* El submódulo **interp1d** es uno de los más convenientes.

In [ ]:
import numpy as np
datos = np.genfromtxt('meteorologia.csv', delimiter=',', names=True)
datos

In [ ]:
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

f = interp1d(datos['Mes'],datos['Temperaturac']) #Interpola una función 1-D, el método se cambia con kind
x = np.linspace(1,12,12)
plt.plot(datos['Mes'],datos['Temperaturac'],'o',x,f(x),'-')
plt.show()

print('Valor real en setiembre: 23 °C \nValor interpolado: ',f(9),'°C')

In [ ]:
interp1d?

In [ ]:
A, nu, k = 10, 4, 2

def f(x, A, nu, k):
    return A * np.exp(-k*x) * np.cos(2*np.pi * nu * x)

xmax, nx = 0.5, 8
x = np.linspace(0, xmax, nx)
y = f(x, A, nu, k)

f_nearest = interp1d(x, y, kind='nearest')
f_linear  = interp1d(x, y)
f_cubic   = interp1d(x, y, kind='cubic')

x2 = np.linspace(0, xmax, 100)
plt.plot(x, y, 'o', label='data points')
plt.plot(x2, f(x2, A, nu, k), label='exact')
plt.plot(x2, f_nearest(x2), label='nearest')
plt.plot(x2, f_linear(x2), label='linear')
plt.plot(x2, f_cubic(x2), label='cubic')
plt.legend()
plt.show()

Ejemplo tomado de [Learning Scientific Programming with Python](https://scipython.com/book/chapter-8-scipy/examples/scipyinterpolateinterp1d/)

### ¿Qué sucede si tenemos ruido en nuestros datos? ¿Podemos interpolar?

La respuesta es sí. Las funciones *spline* son particularmente buenas para estos casos. Usando el *smoothing factor* podemos evitar que el polinomio interpolador pase por todos los puntos, ignorando un poco el ruido.

#### ¿Cómo agregar ruido artificial a mis datos?

Con NumPy, podemos usar el módulo random para añadir ruido.

Supongamos que estamos midiendo una señal sinusoidal.

In [ ]:
import numpy as np

x = np.arange(0,4*np.pi,0.3)
y = np.cos(x)
y

In [ ]:
y_ruido = y + np.random.normal(0, 0.05, y.shape) #Agregarmos ruido simplemente añadiendo datos obtenidos de una distribución normal
y_ruido

In [ ]:
y_ruido2 = y + np.log(1.05) #Agregamos ruido con una función
y_ruido2

In [ ]:
plt.plot(x,y,x,y_ruido,x,y_ruido2)
plt.show()

Ahora vamos a interpolar valores para la señal con ruido, que fue la que medimos en nuestro experimento.

In [ ]:
from scipy.interpolate import UnivariateSpline

plt.plot(x,y_ruido,'o')
plt.show()

fr = UnivariateSpline(x,y,s=0.2) #El smoothing factor distinto de cero hace que no pase por todos los puntos, 'cancelando' el ruido
frx = np.linspace(x.min(),x.max(),100)

In [ ]:
plt.plot(x,y_ruido,'o',frx,fr(frx))
plt.show()

# Módulo scipy.optimize: Optimización

Provee funciones para maximizar o minimizar funciones objetivo y encontrar raíces.



In [ ]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def f(ent):
  x,y = ent
  return x/np.exp(x**2 + y**2) #Función que queremos minimizar

x = np.arange(-2,2,0.1)
y = np.arange(-2,2,0.1)
entt = np.meshgrid(x,y)
z= f(entt)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(entt[0], entt[1], z)
plt.show()

In [ ]:
optimize.minimize?

In [ ]:
ent0 = [0,0]
min = optimize.minimize(f,x0=ent0) #Para el algoritmo debemos dar una condición inicial x0 para todas las variables
min

In [ ]:
min = optimize.minimize(f,x0=ent0,method='Nelder-Mead') #Cambiamos de algoritmo
min

# Módulo scipy.signal: Procesamiento de señales

* Contiene filtros y algunas técnicas de diseño de filtros.
* También cuenta con algunos métodos de interpolación.


In [ ]:
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt

samplerate, data = wavfile.read('pajarito.wav') #Devuelve la cantidad de muestras por segundo y los datos

#Datos tienen forma (número de datos, número de canales)
print('Muestras por segundo: %d \nNúmero de datos: %d\nNúmero de canales: %d' %(samplerate,data.shape[0],data.shape[1]))

In [ ]:
length = data.shape[0] / samplerate #Longitud del audio en segundos
time = np.linspace(0., length, data.shape[0]) #Creamos tantos datos de tiempo como muestras
plt.plot(time, data[:,0]) #Ploteamos solo un canal
plt.show()

Podemos aplicar una transformada de Fourier para ver el dominio de la frecuencia. Esto lo logramos con el módulo scipy.fft.

In [ ]:
from scipy.fft import fft, fftfreq

yf = fft(data[:,0])
xf = fftfreq(data.shape[0], 1/samplerate) #Valores de frecuencia

plt.plot(xf, np.abs(yf)) #Se utiliza el valor absoluto porque porque los valores de yf son complejos
plt.minorticks_on()
plt.show()


In [ ]:
from scipy.signal import butter,sosfilt

#sos =  butter(2,1000,'hp',output='sos',fs=samplerate) #Filtro paso alto de orden 2, Butterworth
sos =  butter(1,0.09,'hp',output='sos') #Filtro paso alto de orden 1, Butterworth
filtered = sosfilt(sos, data[:,0])
plt.plot(time, filtered)
plt.show()

wavfile.write("pajarito_filt.wav", samplerate, filtered.astype(np.int16))

In [ ]:
from scipy.fft import fft, fftfreq

yff = fft(filtered)

plt.plot(xf, np.abs(yff))
plt.minorticks_on()
plt.show()

# Módulo scipy.stats: Estadística

Ofrece clases para representar variables aleatorias con gran variedad de distribuciones de probabilidad. Estas variables aleatorias pueden ser utilizadas para:

* Realizar estadística descriptiva.
* Generar muestras aleatorias que sigan la distribución asociada a la variable aleatoria.

In [ ]:
from scipy import stats

variableDistNormal = stats.norm(1,0.5) #Creamos una variable aleatoria normalmente distribuida (mean, std)
print(variableDistNormal) #Esto retorna un objeto de Python, no un arreglo de muestras
variableDistNormal.mean()

Es posible utilizar estadística descriptiva sobre esta variable aleatoria.

In [ ]:
print('Media: ',variableDistNormal.mean())
print('Mediana: ',variableDistNormal.median())
print('Varianza: ',variableDistNormal.var())

Además, con una variable aleatoria podemos evaluar su función de densidad de probabilidad.

In [ ]:
variableDistNormal.pdf([0,1,2]) #Recibe como parámetro un arreglo de valores donde evaluar la función

In [ ]:
variableDistNormal.cdf([0,1,2]) #Función de distribución acumulada

## Muestras aleatorias de una distribución

Podemos generar muestras aleatorias que sigan la distribución asociada a la variable aleatoria que creamos. La funcion **rvs()** de stats nos permite realizar esta tarea.

In [ ]:
variableDistNormal.rvs(10)

### *stats* permite representar distribuciones continuas, multivariadas y discretas. Posee alrededor de 80 de ellas. Se pueden consultar [aquí](https://docs.scipy.org/doc/scipy/reference/stats.html).

## Estadística descriptiva de un conjunto de datos

Tomemos los datos meteorológicos que utilizamos anteriormente. ¿Qué podemos decir de ellos?

In [ ]:
datos

In [ ]:
stats.describe(datos['Dias_Llovidos']) #Descripción general de los datos

In [ ]:
stats.mode(datos['Humedad']) #La moda, el valor más repetido

También podemos obtener la frecuencia acumulada de los datos.

In [ ]:
cdf = stats.cumfreq(datos['Precipitacionmm'], numbins=4) #numbins es la cantidad de categorías
cdf




# Módulo scipy.sparse: Matrices dispersas
    
## ¿Qué es una matriz dispersa?

* Es una matriz en la cual **la mayoría de sus entradas son cero**.
* Se utilizan en la solución de ecuaciones diferenciales en áreas como la ingeniería, así como en el análisis numérico y la teoría de grafos o redes.
* La representación de estas matrices se puede dar en distintos formatos. *Ejemplo:* COO, CSR, CSC.

$$M = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 \\ 2 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 4 & 0 \\ 0 & 0 & 0 & 3 & 0 \\  0 & 0 & 0 & 0 & 5\end{bmatrix}$$

In [ ]:
from scipy import sparse

ind_f = np.array([0,1,3,2,4]) #Indices de las filas
ind_c = np.array([0,0,3,3,4]) #Indices de las columnas
datos = np.array([1,2,3,4,5]) #Datos
mat = sparse.coo_matrix((datos,(ind_f,ind_c))) #Crea una matriz dispersa en formato COO (COOrdinate)

print(mat)

In [ ]:
mat.toarray() #Imprime la matriz creada como un array, mas sencillo de visualizar

In [ ]:
#Noten que el resultado es una estructura de datos que representa la matriz 
#Estos objetos tienen atributos que podemos extraer

print(mat.shape)
print(mat.size)
print(mat.nnz) #Número de valores que no son cero
print(mat.data)

**find()** devuelve los valores y los índices de los elementos que no son cero en la matriz.

In [ ]:
sparse.find(mat)

También se pueden generar matrices dispersas aleatorias, hacer operaciones como el producto de Kronecker, comprobar si una matriz es de cierto tipo o incluso salvar y cargar una matriz en un archivo formato **.npz**. Más información de este formato [aquí](https://imageio.readthedocs.io/en/stable/format_npz.html).

In [ ]:
A = sparse.rand(3,3,density=0.25, format='coo') #Se crean dos matrices aleatorias de 3x3 con cierta densidad y formato COO
B = sparse.rand(3,3,density=0.4, format='coo')
print(A.toarray(),'\n')
print(B.toarray(),'\n')

In [ ]:
C = sparse.kron(A,B,format='coo') #Calcula el producto de Kronecker entre A y B, lo guarda en formato COO
print(C)
print(C.toarray(),'\n')

In [ ]:
sparse.save_npz('C.npz', C) #Guarda el producto anterior en el archivo C.npz
C_archivo = sparse.load_npz('C.npz') #Carga la matriz de C.npz nuevamente

print(C_archivo)

print(sparse.issparse(C_archivo)) #No comprueba si la matriz es dispersa, comprueba si es un objeto sparse

*sparse* tiene además dos submódulos:

* *csgraph*, algoritmos de grafos basados en representaciones de matrices dispersas.
* *linalg*, álgebra lineal para matrices dispersas.

### Para explorar más de _sparse_, se puede consultar [aquí](https://docs.scipy.org/doc/scipy/reference/sparse.html)

## Grafos

Un grafo es un conjunto de **vértices** (también llamados **nodos**) que se conectan entre sí a través de **aristas**. A las aristas se les puede asignar un valor llamado **peso**.

<center> <img src="https://drive.google.com/uc?export=view&id=1aCxtdof-NX5sXpQHKFb8TL-cNue-7vcu" alt="image info" width="300"/> </center>

Un grafo es una buena representación para sistemas que se basan en redes. Por ejemplo:

* Movilidad entre ciudades.
* Células y sus interacciones.
* Contactos sociales.
* Átomos, moléculas y sus interacciones.
* Procesamiento del lenguaje.

Una de las representaciones de los grafos es a través de matrices. Estas pueden ser tanto densas como dispersas.

Los nodos se ubican en columnas y filas de la matriz, y cada entrada corresponde al peso de la relación entre ambos nodos.

$$M_{grafo} = \begin{bmatrix}
0 & 0 & 2 & 0 & 1 & 1\\ 
0 & 0 & 0 &0  & 3 & 1 \\ 
2 & 0 & 0 & 5 & 3 & 0\\ 
0 & 0 & 5 & 0 & 0 & 0\\ 
1 & 3 & 3 & 0 & 0 & 2\\ 
1 & 1 & 0 & 0 & 2 & 0
\end{bmatrix} $$

Determinemos si esta matriz es densa o dispersa.




In [ ]:
filas = [2,4,5,4,5,0,3,4,2,0,1,2,5,0,1,4]
columnas = [0,0,0,1,1,2,2,2,3,4,4,4,4,5,5,5]
valores = [2,1,1,3,1,2,5,3,5,1,3,3,2,1,1,2]

m_grafo = sparse.coo_matrix((valores,(filas,columnas)), shape=(6,6))

no_cero= m_grafo.nnz #Elementos que no son cero

densidad = no_cero / np.prod(m_grafo.shape) #Elementos que no son cero/total de elementos
print(densidad) #Podemos decir que la matriz es ligeramente dispersa, ya que su densidad es menor al 50%

Teniendo la representación matricial de un grafo, podemos usar sparse para aplicar algoritmos comunes aplicados a los grafos. Entre ellos están:

* Determinar el camino más corto entre nodos (Dijkstra, Bellman-Ford, etc.)
* Búsqueda de anchura o BFS.
* Árbol recubridor mínimo o MST (Kruskal). 

Todos estos algoritmos son importantes en el diseño de redes (cableado telefónico, eléctrico, TV, hidráulica, entre otros).

Utilicemos un algoritmo de sparse para calcular el camino más corto entre el vértice 3 y el vértice 1 de nuestro grafo.

<center> <img src="https://drive.google.com/uc?export=view&id=1aCxtdof-NX5sXpQHKFb8TL-cNue-7vcu" alt="image info" width="300"/> </center>

In [ ]:
#Devuelve la distancia más pequeña entre el nodo 1 y el 3
dist_min=sparse.csgraph.shortest_path(m_grafo, directed=False, indices=1)
print(dist_min)
dist_min[3]